In [3]:
import pandas as pd

sea_level = pd.read_csv("inputs/sealevel.csv")
data = pd.read_csv("inputs/Zip_homes.csv")
filtered_data = sea_level[::3]

# Requirement 1: Limit to 12 rows per year
years = filtered_data['Year'].unique()
limited_rows = []

for year in years:
    year_data = filtered_data[filtered_data['Year'] == year]
    limited_rows.append(year_data.iloc[:12])

filtered_data = pd.concat(limited_rows)

# Transpose the DataFrame
transposed_data = data.T

# If the index (dates) should be the header
transposed_data.reset_index(inplace=True)
new_header = transposed_data.iloc[0]
transposed_data = transposed_data[1:]
transposed_data.columns = new_header
transposed_data.set_index('RegionName', inplace=True)

# Convert 'Year' column in filtered_data to object type
filtered_data['Year'] = filtered_data['Year'].astype(str)

# Merge dataframes on 'RegionName' and 'Year', using an outer join to handle missing data
merged_data = pd.merge(transposed_data, filtered_data, how='outer', left_index=True, right_on="Year")

# Set the 'Year' column as the index
merged_data.set_index('Year', inplace=True)

# Move the row with the index value "2010-01" to the top
merged_data = pd.concat([merged_data.loc[['2010-01']], merged_data.drop('2010-01')])

# Save the merged DataFrame to a new CSV file
merged_data.to_csv('inputs/merged_data.csv')
print(merged_data.head())
print(merged_data.tail())

          10025.0   60657.0  10023.0  60614.0   79936.0  10002.0  60640.0  \
Year                                                                        
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-02  725000.0  349000.0      NaN      NaN  130000.0      NaN      NaN   
2010-03  762500.0  339619.5      NaN      NaN  129900.0      NaN      NaN   
2010-04  725000.0  349000.0      NaN      NaN  129900.0      NaN      NaN   
2010-05  732500.0  334500.0      NaN      NaN  131450.0      NaN      NaN   

          77084.0  94109.0   75070.0  ...  76005.0  81225.0  \
Year                                  ...                     
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-02  119850.0      NaN  194900.0  ...      NaN      NaN   
2010-03  121950.0      NaN  190995.0  ...      NaN      NaN   
2010-04  120000.0      NaN  195000.0  ...      NaN      NaN   
2010-05  120000.0      NaN  194950.0  ...      NaN      NaN   

         TotalWeig

In [10]:
print(filtered_data)

      Year  TotalWeightedObservations  GMSL_noGIA  StdDevGMSL_noGIA  \
0     1993                  327401.31      -38.59             89.86   
3     1993                  297483.19      -42.67             90.75   
6     1993                  327830.00      -36.11             88.74   
9     1993                  331127.31      -39.25             98.10   
12    1993                  328241.09      -36.63             86.34   
...    ...                        ...         ...               ...   
1035  2021                  335568.41       47.10             87.00   
1038  2021                  336371.50       49.00             86.66   
1041  2021                  322835.09       50.89             85.92   
1044  2021                  338042.09       50.49             85.94   
1047  2021                  335065.19       50.07             82.04   

      SmoothedGSML_noGIA  GMSL_GIA  StdDevGMSL_GIA  SmoothedGSML_GIA  \
0                 -38.76    -38.59           89.86            -38.75   
3  

In [9]:
import pandas as pd

sea_level = pd.read_csv("inputs/sealevel.csv")
data = pd.read_csv("inputs/Zip_homes.csv")
filtered_data = sea_level[::3]

# Requirement 1: Limit to 12 rows per year
years = filtered_data['Year'].unique()
limited_rows = []

for year in years:
    year_data = filtered_data[filtered_data['Year'] == year]
    limited_rows.append(year_data.iloc[:12])

filtered_data = pd.concat(limited_rows)

# Transpose the DataFrame
transposed_data = data.T

# If the index (dates) should be the header
transposed_data.reset_index(inplace=True)
new_header = transposed_data.iloc[0]
transposed_data = transposed_data[1:]
transposed_data.columns = new_header
transposed_data.set_index('RegionName', inplace=True)

# Convert 'Year' column in filtered_data to object type
filtered_data['Year'] = filtered_data['Year'].astype(str)

# Merge dataframes on 'RegionName' and 'Year', using an outer join to handle missing data
merged_data = pd.merge(transposed_data, filtered_data, how='outer', left_index=True, right_on="Year")

# Set the 'Year' column as the index
merged_data.set_index('Year', inplace=True)

# Rename the index
merged_data.index = merged_data.index.rename('Date')

# Save the merged DataFrame to a new CSV file
merged_data.to_csv('inputs/merged_data.csv')

print(merged_data.head())

          10025.0   60657.0  10023.0  60614.0   79936.0  10002.0  60640.0  \
Date                                                                        
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-02  725000.0  349000.0      NaN      NaN  130000.0      NaN      NaN   
2010-03  762500.0  339619.5      NaN      NaN  129900.0      NaN      NaN   
2010-04  725000.0  349000.0      NaN      NaN  129900.0      NaN      NaN   
2010-05  732500.0  334500.0      NaN      NaN  131450.0      NaN      NaN   

          77084.0  94109.0   75070.0  ...  76005.0  81225.0  \
Date                                  ...                     
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-02  119850.0      NaN  194900.0  ...      NaN      NaN   
2010-03  121950.0      NaN  190995.0  ...      NaN      NaN   
2010-04  120000.0      NaN  195000.0  ...      NaN      NaN   
2010-05  120000.0      NaN  194950.0  ...      NaN      NaN   

         TotalWeig

In [11]:
import pandas as pd

sea_level = pd.read_csv("inputs/sealevel.csv")
data = pd.read_csv("inputs/Zip_homes.csv")
filtered_data = sea_level[::3]

# Requirement 1: Limit to 12 rows per year
years = filtered_data['Year'].unique()
limited_rows = []

for year in years:
    year_data = filtered_data[filtered_data['Year'] == year]
    limited_rows.append(year_data.iloc[:12])

filtered_data = pd.concat(limited_rows)

# Transpose the DataFrame
transposed_data = data.T

# If the index (dates) should be the header
transposed_data.reset_index(inplace=True)
new_header = transposed_data.iloc[0]
transposed_data = transposed_data[1:]
transposed_data.columns = new_header

# Rename 'RegionName' to 'Year'
transposed_data.rename(columns={'RegionName': 'Year'}, inplace=True)
transposed_data.set_index('Year', inplace=True)

# Format 'Year' column in filtered_data
filtered_data['Year'] = pd.to_datetime(filtered_data['Year'], format='%Y').dt.strftime('%Y-%m')

# Merge dataframes on 'Year', using an outer join to handle missing data
merged_data = pd.merge(transposed_data, filtered_data, how='outer', left_index=True, right_on="Year")

# Set the 'Year' column as the index
merged_data.set_index('Year', inplace=True)

# Rename the index
merged_data.index = merged_data.index.rename('Date')

# Save the merged DataFrame to a new CSV file
merged_data.to_csv('inputs/merged_data.csv')

print(merged_data.head())

          10025.0   60657.0  10023.0  60614.0   79936.0  10002.0  60640.0  \
Date                                                                        
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   

          77084.0  94109.0   75070.0  ...  76005.0  81225.0  \
Date                                  ...                     
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   

         TotalWeig

In [16]:
import pandas as pd
from pandas.tseries.offsets import DateOffset

sea_level = pd.read_csv("inputs/sealevel.csv")
data = pd.read_csv("inputs/Zip_homes.csv")
filtered_data = sea_level[::3]

# Requirement 1: Limit to 12 rows per year
years = filtered_data['Year'].unique()
limited_rows = []

for year in years:
    year_data = filtered_data[filtered_data['Year'] == year]
    limited_rows.append(year_data.iloc[:12])

filtered_data = pd.concat(limited_rows)

# Transpose the DataFrame
transposed_data = data.T

# If the index (dates) should be the header
transposed_data.reset_index(inplace=True)
new_header = transposed_data.iloc[0]
transposed_data = transposed_data[1:]
transposed_data.columns = new_header

# Rename 'RegionName' to 'Year'
transposed_data.rename(columns={'RegionName': 'Year'}, inplace=True)
transposed_data.set_index('Year', inplace=True)

# Format 'Year' column in filtered_data
filtered_data['Year'] = pd.to_datetime(filtered_data['Year'], format='%Y') + filtered_data.groupby('Year').cumcount().apply(lambda x: DateOffset(months=x))
filtered_data['Year'] = filtered_data['Year'].dt.to_period('M').astype(str)

# Ensure there are no duplicate months
filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]

# Set the index of the filtered_data DataFrame
filtered_data.set_index('Year', inplace=True)

# Merge dataframes on 'Year', using an outer join to handle missing data
merged_data = pd.merge(transposed_data, filtered_data, how='outer', left_index=True, right_index=True)

# Rename the index
merged_data.index = merged_data.index.rename('Date')

# Save the merged DataFrame to a new CSV file
merged_data.to_csv('inputs/merged_data.csv')

print(merged_data.head())

/Users/andrewbosland/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimelike.py:1189: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  warnings.warn(


         10025.0  60657.0  10023.0  60614.0  79936.0  10002.0  60640.0  \
Date                                                                     
1993-01      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-02      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-03      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-04      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-05      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

         77084.0  94109.0  75070.0  ...  76005.0  81225.0  \
Date                                ...                     
1993-01      NaN      NaN      NaN  ...      NaN      NaN   
1993-02      NaN      NaN      NaN  ...      NaN      NaN   
1993-03      NaN      NaN      NaN  ...      NaN      NaN   
1993-04      NaN      NaN      NaN  ...      NaN      NaN   
1993-05      NaN      NaN      NaN  ...      NaN      NaN   

         TotalWeightedObservations  GMSL_noGIA  StdDe